# Introduction to pandas

Pandas is a python library to work with and store data in a spreadsheet-like format. Pandas allows us to store big datasets of numbers such as type and token counts of hundreds of texts. This notebook introduces some of the basic concepts and functions from Pandas that we'll need, but there are also great introductions out there:

* https://www.w3schools.com/python/pandas/pandas_intro.asp
* https://www.learndatasci.com/tutorials/python-pandas-tutorial-complete-introduction-for-beginners/
* https://pandas.pydata.org/docs/user_guide/10min.html
* http://www.gregreda.com/2013/10/26/intro-to-pandas-data-structures/

We will work with two basic Pandas data structures:

* Series
* Dataframes

A series is very similar to a list or a dictionary. For instance, we have seen lists of words (tokens) extracted from a document and dictionaries of words and their frequencies:

* List: `['If', 'you', 'use', '``', 'bad', "''", 'to', 'mean', '``', 'good', "''", ',', 'then', ...]`
* Dictionary: `{',': 31, 'the': 24, '.': 21, 'a': 14, 'and': 14, 'to': 13, "'s": 6, 'The': 6, 'I': 6, 'is': 5, ...}`

In a series, you'll get that dictionary as a vertical list: 

```
,     31
the   24
.     21
a     14
and   14
```

Then, with Pandas, you can turn those series into dataframes. Dataframes are like spreadsheets. So, from the dictionary above, we can create a dataframe that looks sort of like this:

| Token       | Count    |
| ----------- | -------- |
| ,           | 31       |
| the         | 24       |
| .           | 21       |
| a           | 14       |
| and         | 14       |


We'll first start with the same process we had in a previous notebook, `02_files_corpora.ipynb`, where we had files and their token, type, sentence, and lexical diversity count. We'll convert those lists and dictionaries into series and then put them into a dataframe. Finally, and to save the information, we'll use a Pandas function to save that dataframe into a csv file. A comma-separated value file is an easy way to store table information in a text-only format.  

We start by importing all the packages we need. Note that we import Pandas as "pd". That allows us to type the Pandas functions with a shorthand, "pd", rather than the full "pandas". 

In [ ]:
import os
import nltk
import numpy
import matplotlib
from nltk.corpus import PlaintextCorpusReader
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk import FreqDist
import pandas as pd

In [ ]:
# This part is the same process we did in the files_corpora notebook
corpus_root = "./small_corpus/"
reviews = PlaintextCorpusReader(corpus_root, '.*', encoding = "utf8")

# store the names of the files in the corpus into a variable, 'fileids'
fileids = reviews.fileids()

# create empty dictionaries for all the information
tokenized_reviews = {}
sents_reviews = {}
token_count = {}
type_count = {}
sent_count = {}

# the for loop goes through each text (raw), tokenizes it (word_tokenize), and saves
# the tokens for each file in the tokenized_reviews dictionary
for fileid in fileids:
    tokens = word_tokenize(reviews.raw(fileid))
    tokenized_reviews[fileid] = tokens
    token_count[fileid] = len(tokenized_reviews[fileid])
    type_count[fileid] = len(set(tokenized_reviews[fileid]))
    sents = sent_tokenize(reviews.raw(fileid))
    sents_reviews[fileid] = sents
    sent_count[fileid] = len(sents_reviews[fileid])

## Dictionaries to series

Now, instead of printing, we convert all the dictionaries into series. Each series has the name of the file as the first column and the information in the second column. So, the tokens_s (tokens series) has "bad_santa.txt" as the first column and the list of tokens in the file as the second column. We create a series for each of the dictionaries that we created above. I have separate cells to show the contents of the first couple of series, but you can print the contents of all the series if you want to see them. (Note: you can also go straight from dictionary to dataframe without having to put into a series first.)

In [ ]:
tokenized_reviews

In [ ]:
sents_reviews

In [ ]:
tokens_s = pd.Series(tokenized_reviews)

In [ ]:
tokens_s

In [ ]:
sents_s = pd.Series(sents_reviews)

In [ ]:
sents_s

In [ ]:
token_count_s = pd.Series(token_count)
type_count_s = pd.Series(type_count)
sent_count_s = pd.Series(sent_count)

## Create the dataframe

Once we have all the series, we can put all the information into a dataframe. We use the pandas function `concat()` to add all the information into one dataframe, using the first column to match it all. 

You'll get a nice-looking table structure. The only thing that is not so useful is that the columns have numbers instead of names, as you see when we first print the df. We can rename the columns with the `column()` function. I first re-set the index to be a number, so that I can name the first column as "fileid".

In [ ]:
df = pd.concat([tokens_s, sents_s, token_count_s, type_count_s, sent_count_s], axis=1)

In [ ]:
df

In [ ]:
df.reset_index(inplace=True)

In [ ]:
df

In [ ]:
df.columns = ["fileid", "tokens", "sentences", "token_count", "type_count", "sentence_count"]

In [ ]:
df

## Operations on the df

Now you can also do operations on this dataframe. For instance, you can calculate the lexical diversity (types/tokens) and add a new column with that information.

In [ ]:
df['lexical_diversity'] = df['type_count'] / df['token_count']

In [ ]:
df

## Calculations on the df

You can calculate things from the dataframe, like the average number of tokens or sentences for the entire corpus. These calculations don't change the dataframe; you simply get information from it.

In [ ]:
print(df["token_count"].mean())

In [ ]:
print(df["sentence_count"].mean())

## Save to csv

One of the most useful things about pandas is that you can save the information to a csv file. You can also read in a csv file and convert it to a dataframe. We are going to save all the information from the original file into the "reviews_all_info.csv" file. Then, we'll read in that file, delete the columns that have the lists of tokens and sentences, and save the new df into a different csv file, "reviews_only_numbers.csv". (Note: you don't really have to create a new dataframe and delete the columns, with saving to a csv, you can also specify which columns you want to save; we are only doing this as an exercise.)

I am first creating a new directory, "output", to put the csv files into. For that, I am using the [`os` library](https://www.w3schools.com/python/module_os.asp) that I imported at the beginning of the notebook. If you get an error in the makedirs cell, it's because the directory already exists, which is fine. 

In [ ]:
os.makedirs("output")

In [ ]:
df.to_csv("./output/reviews_all_info.csv", index=False)

In [ ]:
df_counts_only = pd.read_csv("./output/reviews_all_info.csv")

In [ ]:
df_counts_only

In [ ]:
df_counts_only = df_counts_only.drop(['tokens', 'sentences'], axis=1)

In [ ]:
df_counts_only

In [ ]:
df_counts_only.to_csv("./output/reviews_counts_only.csv", index=False)